In [1]:
import data_utils
import os
import rasterio

In [2]:
# Setting the master folder (Which has subfolders image and landslide_data)

master_folder = 'LS_machine_learning_1region'
images_in_master = master_folder + '/Image'
landslides_in_master = master_folder + '/Landslide_Data'

# Output paths
multiband_outputs_folder = 'Processed_Data/Multibands'
mosaics_outputs_folder = 'Processed_Data/Mosaics'
image_patch_outputs_folder = 'Processed_Data/image_patches'
shapefile_patch_outputs = 'Processed_Data/shape_patches'
labels_yolo_path = 'Processed_Data/labels'

# Map from region to desired shapefile

region_to_shapefile={'1987 Sichuan pre-earthquake':landslides_in_master + '/1987 Sichuan rainstorm/preeqbefore1987.shp'}

In [3]:
regions = [os.path.splitext(filename)[0] for filename in os.listdir(images_in_master) if not filename.startswith('.')]
regions_path = [images_in_master + '/' +  os.path.splitext(filename)[0] for filename in os.listdir(images_in_master) if not filename.startswith('.')]


In [14]:
for region in regions:

    # Create folders with region names in Multibands folder
    # Check if the directory exists
    if not os.path.exists(multiband_outputs_folder + '/' + region):
        # If it doesn't exist, create it
        os.makedirs(multiband_outputs_folder + '/' + region)
        
    
    # Create folders with region names in Mosaics folder
    # Check if the directory exists
    if not os.path.exists(mosaics_outputs_folder + '/' + region):
        # If it doesn't exist, create it
        os.makedirs(mosaics_outputs_folder + '/' + region)

    areas_in_region = [os.path.splitext(filename)[0] for filename in os.listdir(images_in_master + '/' + region) if not filename.startswith('.')]
    
    # In each region, first form a multi band tif for each area.
    for area in areas_in_region:
        #print(area)
        area_folder_path = images_in_master + '/' + region + '/' + area
        #print(area_folder_path)

        # TIF band files in an area folder:
        band_names = [bandname for bandname in os.listdir(area_folder_path) if bandname.startswith(area) and bandname.lower().endswith('.tif')]
        #print(band_names)

        output_file_path = multiband_outputs_folder + '/' + region + '/' + area + '_Multiband.TIF'
        print(output_file_path)

        data_utils.combine_bands_to_single_tif(bands_folder = area_folder_path, band_files = band_names, output_file = output_file_path)
        
    print('multiband area creation done')
    ####### AFTER ALL AREA MULTIBANDS ARE GENERATED:
    # Now combine to form one big mosaic (join multiple area multiband tifs) for a region 

    # Get list of raster multibands of multiple areas belonging to this region
    multiband_region_folder = multiband_outputs_folder + '/' + region 
    region_multiband_list_paths = [multiband_region_folder + '/' + filename for filename in os.listdir(multiband_region_folder) if filename.lower().endswith('.tif')]
    
    # Path to save mosaic
    output_mosaic_region = mosaics_outputs_folder + '/' + region + '/' + region + '_mosaic.TIF'

    # Create the big mosaic
    data_utils.generate_mosiac_from_raster_list(raster_list = region_multiband_list_paths[1:3], output_path = output_mosaic_region)
    
    # We first obtain the right shapefile to use for the region
    desired_shapefile = region_to_shapefile[region]

    # We generate patches for the region mosaic file and shapefile
    data_utils.generate_patches_from_shapefile_and_raster_patches(input_tif = output_mosaic_region, input_shapefile = desired_shapefile, output_directory_images = image_patch_outputs_folder,output_directory_shapepatches = shapefile_patch_outputs, patch_size=(640, 640))
    
    ############# Preparing patches from mosaic and shapefile
    
    # We first obtain the right shapefile to use for the region
    desired_shapefile = region_to_shapefile[region]

    # We generate patches for the region mosaic file and shapefile
    data_utils.generate_patches_from_shapefile_and_raster_patches(input_tif = output_mosaic_region, input_shapefile = desired_shapefile, output_directory_images = image_patch_outputs_folder,output_directory_shapepatches = shapefile_patch_outputs,region_name = region, patch_size=(640, 640))
    
    ############# Preparing labels from shapefiles
    # We iterate through all shapefiles to generate yolo labels as .txt files

    # Obtain list of shapefiles in the shapefile_patch_outputs folder corresponding to this region
    shapefile_patch_list = [bandname for bandname in os.listdir(shapefile_patch_outputs) if bandname.startswith(region) and bandname.lower().endswith('.shp')]
    
    for shapefile_patch in shapefile_patch_list:
        shapefile_patch_path =  shapefile_patch_outputs + '/' + shapefile_patch
        labelfile_desired_path = labels_yolo_path + '/' + shapefile_patch[:-4] + '.txt'
        data_utils.generate_labels(shapefile_path = shapefile_patch_path,  target_label_path = labelfile_desired_path)


Processed_Data/Multibands/1987 Sichuan pre-earthquake/LE07_L1TP_130038_19991209_20170215_01_T1_Multiband.TIF
Processed_Data/Multibands/1987 Sichuan pre-earthquake/LT05_L1TP_129038_19860731_20170221_01_T1_Multiband.TIF
Processed_Data/Multibands/1987 Sichuan pre-earthquake/LT05_L1TP_130038_19860807_20170221_01_T1_Multiband.TIF
Processed_Data/Multibands/1987 Sichuan pre-earthquake/LT05_L1TP_130038_19870725_20170320_01_T1_Multiband.TIF
Processed_Data/Multibands/1987 Sichuan pre-earthquake/LE07_L1TP_130038_20080405_20161229_01_T1_Multiband.TIF
Processed_Data/Multibands/1987 Sichuan pre-earthquake/LT05_L1TP_130038_20070918_20161111_01_T1_Multiband.TIF
Processed_Data/Multibands/1987 Sichuan pre-earthquake/LT05_L1TP_129038_19870920_20170418_01_T1_Multiband.TIF
Processed_Data/Multibands/1987 Sichuan pre-earthquake/LT05_L1TP_129038_20070506_20161115_01_T1_Multiband.TIF
Processed_Data/Multibands/1987 Sichuan pre-earthquake/LE07_L1TP_130039_20080405_20161229_01_T1_Multiband.TIF
Processed_Data/Mult

DatasetIOShapeError: Dataset indexes and destination buffer are mismatched

# Preparing patches from mosaic and shapefile

In [7]:
# We first obtain the right shapefile to use for the region
desired_shapefile = region_to_shapefile[region]

# We generate patches for the region mosaic file and shapefile
data_utils.generate_patches_from_shapefile_and_raster_patches(input_tif = output_mosaic_region, input_shapefile = desired_shapefile, output_directory_images = image_patch_outputs_folder,output_directory_shapepatches = shapefile_patch_outputs,region_name = region, patch_size=(640, 640))

/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/vihaan/miniforge3/envs/

/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/vihaan/miniforge3/envs/

/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/vihaan/miniforge3/envs/

/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/vihaan/miniforge3/envs/

/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/geopandas/io/file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/vihaan/miniforge3/envs/

...

/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a,

/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. 

/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix wi

/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. 

/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
/Users/vihaan/miniforge3/envs/geek/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotran

# Preparing labels from shapefile



In [5]:
# We iterate through all shapefiles to generate yolo labels as .txt files

# Obtain list of shapefiles in the shapefile_patch_outputs folder corresponding to this region
shapefile_patch_list = [bandname for bandname in os.listdir(shapefile_patch_outputs) if bandname.startswith(region) and bandname.lower().endswith('.shp')]

print(shapefile_patch_list)

['1987 Sichuan pre-earthquake_patch_14720_9600.shp', '1987 Sichuan pre-earthquake_patch_15360_7040.shp', '1987 Sichuan pre-earthquake_patch_14720_5760.shp', '1987 Sichuan pre-earthquake_patch_14080_5760.shp', '1987 Sichuan pre-earthquake_patch_640_3840.shp', '1987 Sichuan pre-earthquake_patch_14080_9600.shp', '1987 Sichuan pre-earthquake_patch_6400_13440.shp', '1987 Sichuan pre-earthquake_patch_11520_8960.shp', '1987 Sichuan pre-earthquake_patch_3840_3200.shp', '1987 Sichuan pre-earthquake_patch_15360_10880.shp', '1987 Sichuan pre-earthquake_patch_7040_11520.shp', '1987 Sichuan pre-earthquake_patch_10880_10240.shp', '1987 Sichuan pre-earthquake_patch_0_11520.shp', '1987 Sichuan pre-earthquake_patch_8960_12800.shp', '1987 Sichuan pre-earthquake_patch_4480_8960.shp', '1987 Sichuan pre-earthquake_patch_12160_640.shp', '1987 Sichuan pre-earthquake_patch_1920_3840.shp', '1987 Sichuan pre-earthquake_patch_12160_8320.shp', '1987 Sichuan pre-earthquake_patch_12800_10880.shp', '1987 Sichuan pre

In [7]:
for shapefile_patch in shapefile_patch_list:
    shapefile_patch_path =  shapefile_patch_outputs + '/' + shapefile_patch
    labelfile_desired_path = labels_yolo_path + '/' + shapefile_patch[:-4] + '.txt'
    data_utils.generate_labels(shapefile_path = shapefile_patch_path,  target_label_path = labelfile_desired_path)